## 과제

영어 불용어(사용할 필요가 없는 단어/중요하지 않는 단어 - 부정관사, 정관사, 등) 리스트를 참고하여 입력 문장에서 불용어를 제거한 SVM 기반 스팸 메일 필터를 구현하시오.


In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

print("영어 불용어 개수:", len(nltk.corpus.stopwords.words('english')))
# print(nltk.corpus.stopwords.words('english')[:10])

stopwords = nltk.corpus.stopwords.words('english')
# 불용어라고 판단되는 2가지 데이터 추가하였습니다.
stopwords.append('...')
stopwords.append('..')
print(stopwords)

In [ ]:
import numpy as np

file_path = "/gdrive/My Drive/ml_colab/week4/SMSSpamCollection.dat"

# 파일 읽기
x_real_data, x_data, y_data = [], [], []
with open(file_path,'r',encoding='utf8') as inFile:
  lines = inFile.readlines()
  
lines = lines[:100]  
  
for line in lines:
  line = line.strip().split('\t')
  sentence, label = line[1], line[0] # 문장은 sentence / 정답은 label
  x_real_data.append(sentence)
  y_data.append(label)

print(x_real_data)

# 불용어 제거
for i, document in enumerate(x_real_data):
    clean_words = [] 
    for word in nltk.tokenize.word_tokenize(document): 
        if word not in stopwords:
            clean_words.append(word)
    x_data.append(' '.join(clean_words))    
print(x_data)

print("x_data의 개수 : " + str(len(x_data)))
print("y_data의 개수 : " + str(len(y_data)))

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'U dun say so early hor... U c already then say...', "Nah I don't think he goes to usf, he lives around here though", "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, ￡1.50 to rcv", 'Even my brother is not like to speak with me. They treat me like aids patent.', "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune", 'WINNER!! As a valued network customer you have been selected to receivea ￡900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', 'Had your mobile 

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

label2index_dict = {'spam':0, 'ham':1}

indexing_x_data, indexing_y_data = [], []

for label in y_data:
  indexing_y_data.append(label2index_dict[label])

tokenizer.fit_on_texts(x_data)

indexing_x_data = tokenizer.texts_to_sequences(x_data)    

print("x_data indexing 하기 전 : " + str(x_data[0]))
print("x_data indexing 하기 후 : " + str(indexing_x_data[0]))
print("y_data indexing 하기 전 : " + str(y_data[0]))
print("y_data indexing 하기 후 : " + str(indexing_y_data[0]))

x_data indexing 하기 전 : Go jurong point , crazy .. Available bugis n great world la e buffet ... Cine got amore wat ...
x_data indexing 하기 후 : [11, 197, 198, 199, 200, 201, 33, 49, 202, 203, 79, 204, 205, 19, 206, 80]
y_data indexing 하기 전 : ham
y_data indexing 하기 후 : 1


In [ ]:
from sklearn.svm import SVC

# 문장의 길이를 max_length으로 맞춰 변환
max_length = 60
for index in range(len(indexing_x_data)):
  length = len(indexing_x_data[index])
  
  if(length > max_length):
    indexing_x_data[index] = indexing_x_data[index][:max_length]
  elif(length < max_length):
    indexing_x_data[index] = indexing_x_data[index] + [0]*(max_length-length) # 늘어난 길이는 0으로 채움
    
    
# 전체 데이터를 9:1의 비율로 나누어 학습 및 평가 데이터로 사용
number_of_train = int(len(indexing_x_data)*0.9)

train_x = indexing_x_data[:number_of_train]
train_y = indexing_y_data[:number_of_train]
test_x = indexing_x_data[number_of_train:]
test_y = indexing_y_data[number_of_train:]

print("train_x의 개수 : " + str(len(train_x)))
print("train_y의 개수 : " + str(len(train_y)))
print("test_x의 개수 : " + str(len(test_x)))
print("test_y의 개수 : " + str(len(test_y)))

# linear kernel
svm = SVC(kernel='linear', C=1e10)
svm.fit(train_x, train_y)

train_x의 개수 : 90
train_y의 개수 : 90
test_x의 개수 : 10
test_y의 개수 : 10


SVC(C=10000000000.0, kernel='linear')

In [ ]:
predict = svm.predict(test_x)

correct_count = 0
for index in range(len(predict)):
  if(test_y[index] == predict[index]):
    correct_count += 1
    
accuracy = 100.0*correct_count/len(test_y)

print("Accuracy: " + str(accuracy))

index2label = {0:"spam", 1:"ham"}

test_x_word = tokenizer.sequences_to_texts(x_real_data[number_of_train:])

for index in range(len(test_x_word)):
  print()
  print("문장 : ", test_x_word[index])
  print("정답 : ", index2label[test_y[index]])
  print("모델 출력 : ", index2label[predict[index]])

Accuracy: 90.0

문장 :  
정답 :  ham
모델 출력 :  ham

문장 :  
정답 :  ham
모델 출력 :  ham

문장 :  
정답 :  ham
모델 출력 :  ham

문장 :  
정답 :  spam
모델 출력 :  spam

문장 :  
정답 :  ham
모델 출력 :  ham

문장 :  
정답 :  spam
모델 출력 :  spam

문장 :  
정답 :  ham
모델 출력 :  ham

문장 :  
정답 :  ham
모델 출력 :  ham

문장 :  
정답 :  ham
모델 출력 :  spam

문장 :  
정답 :  ham
모델 출력 :  ham
